# Inputs - representación de las palabras

Lo primero que tenemos que ver es cómo le introducimos una frase al transformer

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_inputs.png" alt="Encoder inputs" style="width:425px;height:626px;">
</div>

# Representación de las palabras

Como hemos visto en el tema de redes convolucionales, las imágenes se representan como la cantidad de rojo, verde y azul de cada pixel. Esa cantidad es un número que varía de entre 0 y 255. Es decir, para representar una imagen necesitamos hacerlo mediante números.

Con el lenguaje pasa igual, para poder procesarlo y realizar predicciones o generar texto, necesitamos poder representarlo mediante números. Para ello se utilizan los `token`s

## Tokens

Para explicar qué son los `token`s, primero veámoslo con un ejemplo práctico, vamos a usar el tokenizador de BERT

BERT es un modelo grande de lenguaje creado por Google. Para usar su tokenizador vamos a usar la librería de huggingface

La empresa [huggingface](https://huggingface.co) se hizo muy popular gracias a su librería [transformers](https://huggingface.co/docs/transformers), ya que hizo que fuese muy sencillo usarlos gracias a su API. Ahora no solo podemos usar transformers, sino casi cualquier modelo, ya que se autodefinen como el GitHub del deep learning, así que casi todos los nuevos modelos se publican también en huggingface, donde los puedes usar con su librería, ver su documentación e incluso probar online gracias a sus `spaces`

En este curso no vamos a explicar hugginface, pero es bueno que la conozcas, ya que es un muy buen recurso

Primero vamos a importar las librerías necesarias

In [1]:
from transformers import BertTokenizer, BertModel
import torch

Cargamos el modelo BERT

In [2]:
model = BertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Seguramente obtendrás un warning como yo, ya que estamos importando los pesos de `bert-base-multilingual-cased` que fue entrenado con más capas de las que aparecen en el modelo `BertModel`. Si fuésemos a usar el modelo para lo que fue entrenado y dio como resultado los pesos `bert-base-multilingual-cased` deberíamos importar las capas extra, pero como no es nuestro caso podemos ignorar el warning

Ahora cargamos su tokenizador

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Obtenemos los `token`s de la palabra `hello`

In [4]:
input_ids = tokenizer.encode("hello", add_special_tokens=True)
input_ids

[101, 61694, 10133, 102]

Vamos a analizar los `ids` o `token`s.

Por cómo está implementado BERT siempre espera que cada frase empieze con el token `[CLS]` (`101`) y que termine con el token `[SEP]` (`102`), por lo que el tokenizador de BERT los añade automáticamente

Por otro lado los tokens `61694` y `10133` son los tokens generados por dividir la palabra `hello` en dos subpalabras, que son

In [5]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
tokens

['[CLS]', 'hell', '##o', '[SEP]']

BERT ha dividido la palabra `hello` en `hell` y `##o`.

Cuando se crea el vocabulario que se quiere que tenga nuestro modelo, se hacen descomposiciones de palabras en otras más sencillas, a estas se les llaman `token`s

De modo que no habrá un token `perro` y otro `perra` sino que habrá un token `perr`, otro `##o` y otro `##a` (las dobles almohadillas indican que son el final de una palabra).

Puedes pensar, pero `perro` y `perra` son dos tokens y `perr`, `##o` y `##a` son tres tokens, y es verdad, pero ahora podemos añadir el token `herman` y juntarlo con los tokens ya creados `##o` y `##a`, y añadir el token `cuñad` y volver a juntarlo con los tokens ya creados `##o` y `##a`. De esta manera hemos pasado de tener 6 tokens `perro`, `perra`, `hermano`, `hermana`, `cuñado` y `cuñada` a solo 5 con `perr`, `herman`, `cuñad`, `##o` y `##a`. Hemos ahorrado un token, y a medida que añadamos más palabras terminadas en `o` o `a` seguiremos reduciendo el número de tokens.

Esta técnica y otras de dividir las palabras en otras más sencillas se llama tokenización, lo que produce un vocabulario más pequeño y así poder usar modelos más pequeños